In [21]:
import requests
import pandas as pd

In [22]:
# Esta funcion imprime las 10 monedas volatiles con mas capitalizacion y luego las 10 monedas estables.
def get_top_coins():
    stable = []
    volatil = []

    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()

    for coin in data:
        if coin['current_price'] > 1 or 0.99 > coin['current_price'] and len(volatil) < 10:
            volatil.append(coin['id'])
        elif 1.1 > coin['current_price'] > 0.97 and len(stable) < 10:
            stable.append(coin['id'])
        else:
            break
        
    return stable,volatil

stable, volatil = get_top_coins()
print(stable)
print(volatil)

['tether', 'usd-coin']
['bitcoin', 'ethereum', 'binancecoin', 'ripple', 'staked-ether', 'cardano', 'dogecoin', 'solana', 'tron', 'polkadot']


In [27]:
# Utiliza una lista de nombres de criptomonedas y a traves de la api de coingecko extrae la informacion de cada una de ellas para luego de un ETL guardar 
# la informacion en un archivo csv.
def get_historical_data(coinnamelist):
    for coin in coinnamelist:
        url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart"
        params = {
            'vs_currency': 'usd',
            'days': '2160',
            'interval': 'daily'
        }
    
        response = requests.get(url, params=params)
    
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)
            df['market_caps'] = df['market_caps'].apply(lambda x: int(x[1]))
            df['total_volumes'] = df['total_volumes'].apply(lambda x: int(x[1]))
            df['Date'] = df['prices'].apply(lambda x: pd.to_datetime(x[0], unit='ms'))
            df['prices'] = df['prices'].apply(lambda x: int(x[1]))
            df.rename(columns={'prices': 'Price_usd','total_volumes':'Volume','market_caps':'Market_cap'}, inplace=True)
            df = df.groupby(df["Date"].dt.to_period("M")).tail(1)
            df.to_csv(f'./Data/{coin}.csv',index=False)
        else:
            print(f"Error al obtener los datos de {coin}:", response.status_code)


In [28]:
get_historical_data(volatil)

In [15]:
df.head()

,prices,market_caps,total_volumes,Date
0,3898,64629619385,942656241,2017-09-19
1,3858,63957210363,676074373,2017-09-20
2,3628,60152757874,913652257,2017-09-21
3,3622,60061814860,767837732,2017-09-22
4,3763,62411285738,578068211,2017-09-23


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2161 entries, 0 to 2160
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Prices_usd   2161 non-null   int64         
 1   Market_caps  2161 non-null   int64         
 2   Volume       2161 non-null   int64         
 3   Date         2161 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 67.7 KB


In [42]:
url = 'https://api.coingecko.com/api/v3/global'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

In [43]:
df_global = pd.DataFrame(data['data'])

In [44]:
df_global

,active_cryptocurrencies,upcoming_icos,ongoing_icos,ended_icos,markets,total_market_cap,total_volume,market_cap_percentage,market_cap_change_percentage_24h_usd,updated_at
btc,10135,0,49,3376,832,4.175184e+07,1.241547e+06,46.620457,-0.835553,1692726230
eth,10135,0,49,3376,832,6.635486e+08,1.973150e+07,18.139543,-0.835553,1692726230
ltc,10135,0,49,3376,832,1.666780e+10,4.956391e+08,NaN,-0.835553,1692726230
bch,10135,0,49,3376,832,5.905055e+09,1.755947e+08,NaN,-0.835553,1692726230
bnb,10135,0,49,3376,832,5.251224e+09,1.561521e+08,2.930481,-0.835553,1692726230
...,...,...,...,...,...,...,...,...,...,...
usdc,10135,0,49,3376,832,NaN,NaN,2.395684,-0.835553,1692726230
steth,10135,0,49,3376,832,NaN,NaN,1.254424,-0.835553,1692726230
ada,10135,0,49,3376,832,NaN,NaN,0.821666,-0.835553,1692726230
doge,10135,0,49,3376,832,NaN,NaN,0.796787,-0.835553,1692726230


In [45]:
df_global['market_cap_percentage'].sum()

83.87693151452041

In [46]:
df_global['updated_at'] = df_global['updated_at'].apply(lambda x: pd.to_datetime(x, unit='ms'))

In [47]:
df_global.head()

,active_cryptocurrencies,upcoming_icos,ongoing_icos,ended_icos,markets,total_market_cap,total_volume,market_cap_percentage,market_cap_change_percentage_24h_usd,updated_at
btc,10135,0,49,3376,832,4.175184e+07,1.241547e+06,46.620457,-0.835553,1970-01-20 14:12:06.230
eth,10135,0,49,3376,832,6.635486e+08,1.973150e+07,18.139543,-0.835553,1970-01-20 14:12:06.230
ltc,10135,0,49,3376,832,1.666780e+10,4.956391e+08,NaN,-0.835553,1970-01-20 14:12:06.230
bch,10135,0,49,3376,832,5.905055e+09,1.755947e+08,NaN,-0.835553,1970-01-20 14:12:06.230
bnb,10135,0,49,3376,832,5.251224e+09,1.561521e+08,2.930481,-0.835553,1970-01-20 14:12:06.230


In [20]:
df.to_csv(f'./Data/global.csv')